In [2]:
import pandas as pd
import time
import urllib3
import json
# Cargar y limpiar el DataFrame
archivo_filename = "C:\\xampp\\htdocs\\datalpine\\resources\\db\\Scrapining\\ciudades\\puebla\\clean\\septiembre_2024.csv"
df = pd.read_csv(archivo_filename)
df['Estado'] = df['Estado'].str.strip().str.lower()
df['Municipio'] = df['Municipio'].str.strip().str.lower()
df['Colonia'] = df['Colonia'].str.strip().str.lower()
df['Colonia_Estandarizada'] = df['Colonia_Estandarizada'].str.strip().str.lower()
# Función de geocodificación usando la API de ArcGIS
def geocode_arcgis(address):
    url = "https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates"
    params = {
        "f": "json",
        "outFields": "*",
        "maxLocations": 1,
        "singleLine": address,
        "outSr": "4326",
        "forStorage": "false"
    }
    try:
        http = urllib3.PoolManager()
        response = http.request('GET', url, fields=params)
        data = response.data.decode('utf-8')
        if data:
            data = json.loads(data)
            if data['candidates']:
                location = data['candidates'][0]['location']
                return location['y'], location['x']
    except urllib3.exceptions.RequestError as e:
        print(f"Error al geocodificar {address}: {e}")
    return None, None  # En caso de error o sin resultados válidos
# Aplicar la geocodificación y guardar en tiempo real
latitudes = []
longitudes = []
for idx, row in df.iterrows():
    direccion_completa = f"{row['Colonia_Estandarizada']}, {row['Municipio']}, {row['Estado']}, México"
    lat, lon = geocode_arcgis(direccion_completa)
    # Si la geocodificación falla, intenta con la colonia oficial
    if lat is None and lon is None:
        direccion_alternativa = f"{row['Colonia']}, {row['Municipio']}, {row['Estado']}, México"
        lat, lon = geocode_arcgis(direccion_alternativa)
    latitudes.append(lat)
    longitudes.append(lon)
    df.loc[idx, 'Latitud'] = lat
    df.loc[idx, 'Longitud'] = lon
    df.to_csv("geocodificado_sep_puebla.csv", index=False, mode='a', header=False)
    time.sleep(0.5)  # Tiempo de espera para evitar sobrecarga en la API
print("Geocodificación completa y guardada en archivo ")

KeyError: 'Colonia_Estandarizada'

In [4]:
import pandas as pd
import time
import urllib3

# Cargar y limpiar el DataFrame
archivo_filename = "C:\\xampp\\htdocs\\datalpine\\resources\\db\\Scrapining\\ciudades\\puebla\\clean\\septiembre_2024.csv"
df = pd.read_csv(archivo_filename)
df['Estado'] = df['Estado'].str.strip().str.lower()
df['Municipio'] = df['Municipio'].str.strip().str.lower()
df['Colonia'] = df['Colonia'].str.strip().str.lower()
df['Colonia_Estandarizada'] = df['Colonia_Estandarizada'].str.strip().str.lower()
df['direccion_completa'] = df['Colonia_Estandarizada'] + ', ' + df['Municipio'] + ', ' + df['Estado'] + ', México'

# Función de geocodificación usando la API de ArcGIS
def geocode_arcgis(address):
    url = "https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates"
    params = {
        "f": "json",
        "outFields": "*",
        "maxLocations": 1,
        "singleLine": address,
        "outSr": "4326",
        "forStorage": "false"
    }
    
    try:
        http = urllib3.PoolManager()
        response = http.request('GET', url, fields=params)
        data = response.data.decode('utf-8')
        
        if data:
            import json
            data = json.loads(data)
            if data['candidates']:
                location = data['candidates'][0]['location']
                return location['y'], location['x']
            else:
                return None, None
        else:
            return None, None
    except urllib3.exceptions.RequestError as e:
        print(f"Error al geocodificar {address}: {e}")
        return None, None

# Aplicar la geocodificación y guardar en tiempo real
for idx, row in df.iterrows():
    lat, lon = geocode_arcgis(row['direccion_completa'])
    # Si la geocodificación falla, intenta con la colonia oficial
    if lat is None and lon is None:
        lat, lon = geocode_arcgis(row['Colonia'] + ', ' + row['Municipio'] + ', ' + row['Estado'] + ', México')
    df.loc[idx, 'Latitud'] = lat
    df.loc[idx, 'Longitud'] = lon
    df.to_csv("geocodificado_sep_puebla.csv", index=False, mode='a', header=False)
    time.sleep(0.5)  # Tiempo de espera para evitar sobrecarga en la API

print("Geocodificación completa y guardada en archivo_geocodificado_arcgis.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\xampp\\htdocs\\datalpine\\resources\\db\\Scrapining\\ciudades\\puebla\\clean\\septiembre_2024.csv'